In [67]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotVisibleException
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
from pandas.io import sql
import os
import pandas as pd
# from decouple import config
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
from PIL import Image
from matplotlib import cm
import re
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime 

from selenium import webdriver
from tqdm import tqdm
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized"); # chrome 브라우저를 최대한 크기로 옵션을 주는 것

# Chrome 실행
browser = webdriver.Chrome('./chromedriver', options=options)


In [68]:
# 경돼 유튜브 이동
browser.get("https://www.youtube.com/channel/UCFa0g-8D8hYuHvdIh6XQCOg")

# 동영상탭 클릭
browser.find_element_by_xpath('//*[@id="tabsContent"]/tp-yt-paper-tab[2]/div').click()

# # 스크롤 다운
# body = browser.find_element_by_css_selector('body')
# for i in range(32):
#     body.send_keys(Keys.PAGE_DOWN)
#     time.sleep(1)

In [ ]:
youtube_dict = {"제목":[], "조회수":[], 
                "영상_게시날짜":[], "좋아요":[], 
                "싫어요":[], "개요":[], "태그":[],
                "영상길이":[],"댓글수":[],"댓글":[]}

# 영상은 총 596개
# 제목, 해시태그, 조회수, 영상 게시날짜, 좋아요, 싫어요, 개요란, 영상길이, 댓글수, 댓글
for i in tqdm(range(1, 596)):
    # 영상 클릭
    browser.find_element_by_xpath(f'/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer[{str(i)}]/div[1]/div[1]/div[1]/h3/a').click()
    
    # 광고 시간 
    time.sleep(10)
    
    # 영상 멈추기
    browser.find_element_by_xpath('//*[@id="movie_player"]/div[1]/video').click()
    
    # 영상 제목
    name = browser.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text
    youtube_dict["제목"].append(name)
    time.sleep(1)
    
    # 조회수
    watch = browser.find_element_by_xpath('//*[@id="count"]/ytd-video-view-count-renderer/span[1]').text
    watch = int("".join(re.findall("[0-9]{1,}",watch)))
    youtube_dict["조회수"].append(watch)
    time.sleep(1)
    
    # 영상 게시일
    day = browser.find_element_by_xpath('//*[@id="date"]/yt-formatted-string').text
    day = day.replace(" ", "")[:-1]
    youtube_dict["영상_게시날짜"].append(day)
    time.sleep(1)
    
    # page_source
    # 좋아요 수와 싫어요 수를 가져오기 위해 사용
    html0 = browser.page_source
    html = BeautifulSoup(html0,'html.parser')

    # 좋아요 수
    # 좋아요 막아 놓은 곳이 있어 try문 사용
    try:
        good = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text
        good = int(good)
        youtube_dict["좋아요"].append(good)
    except:
        youtube_dict["좋아요"].append("")
    time.sleep(1)

    # 싫어요 수
    # 싫어요 막아 놓은 곳이 있어 try문 사용
    try:
        hate = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text
        hate = int(hate)
        youtube_dict["싫어요"].append(hate)
    except:
        youtube_dict["싫어요"].append("")
    
    time.sleep(1)
    
    # 개요
    outline_test = html.find("yt-formatted-string", {"class":"content style-scope ytd-video-secondary-info-renderer"}).text    
    # hash_tag가 안달려있는 영상이 있기때문에 오류 방지
    try:
        hash_tag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
        youtube_dict["태그"].append(hash_tag)
    except Exception as e:
        youtube_dict["태그"].append("")
        
    
    if hash_tag:
        try:
            outline = outline_test.replace(hash_tag, "")
            youtube_dict["개요"].append(outline)
        except:
            youtube_dict["개요"].append("")
    else:
        try:
            youtube_dict["개요"].append(outline_test)
        except:
            youtube_dict["개요"].append("")
    time.sleep(1)
    
    # 영상길이
    video_length = html.find("span", {"class":"ytp-time-duration"}).text
    youtube_dict["영상길이"].append(video_length)
    time.sleep(1)
    
    
    # 스크롤
    # 세로로 긴 영상 때문에 필요
    body = browser.find_element_by_css_selector('body')
    for i in range(3):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
    html0 = browser.page_source
    html = BeautifulSoup(html0,'html.parser')
    
    # 댓글 수
    try:
        comment_num = html.find('yt-formatted-string', {"class":"count-text style-scope ytd-comments-header-renderer"}).text
        comment_num = int(re.findall("[0-9]{1,}", comment_num)[0])
        youtube_dict["댓글수"].append(comment_num)
    except:
        youtube_dict["댓글수"].append("")
    time.sleep(1)
    
        
    # 댓글 lst
    comment_lst = []
    # 1부터
    try:
        comments = html.findAll('yt-formatted-string', {"id":"content-text", "class":"style-scope ytd-comment-renderer"})
        for i in comments:
            comment = i.text
            comment_lst.append(comment)
    except:
        comment_lst.append("")
            
    comment_lst = "\z".join(comment_lst)
    youtube_dict["댓글"].append(comment_lst)

    browser.get('https://www.youtube.com/channel/UCFa0g-8D8hYuHvdIh6XQCOg/videos')
    
    
#     return name, watch, day, good, hate, video_length, comment_num, comment_lst

In [137]:
pd.DataFrame(youtube_dict)

,제목,조회수,영상_게시날짜,좋아요,싫어요,개요,태그,영상길이,댓글수,댓글
0,출근 전 벤치프레스 100kg,1507,2021.5.27,26,3,,,0:41,13,아니 100키로 몇번을 하는거야 ㄷㄷㄷ\z20kg바 맞아여? 출근전 운동진짜..리스...
1,궁금했던 것 물어보면 딱 좋은 타이밍!,1852,2021.5.14,64,2,여러분 안녕하세요 경돼입니다.\n\n모두\n\nBe more positive!\n\...,,0:06,17,"진짜 대박이까지만 보고 껐습니다 죄송합니다\z경돼님 다른건 몰라도 정직 ,근면 ,..."
2,비아냥 속에서도 묵묵히 최선을 다할 것.,5347,2021.4.25,126,6,벚꽃슈레딩 Day25,,0:12,63,경돼형 구독한지 대략3년..드디어 슈레딩 성공을 보다니..\z취준생 시절에 보기 시...
3,"78kg, 썸머슈레딩(다이어트) 시작, 이건 못참지",4393,2021.3.13,99,1,또머슈레딩\n,#썸머슈레딩 #다이어트 #직장인,0:30,53,내게 경돼의 썸머 슈뤠딩이란... \n나 93년 국민학교 여름방학 일기장 항상 못써...
4,아침 저녁으로 운동하기!,2787,2021.3.7,74,2,"여러분 안녕하세요 경돼입니다.\n\n이제 운동을, 하루에 2번씩 넣어주고 있습니다....",,6:58,41,3:01 구독자 감소 point\z와 몸 또 만들었네요.. 대박~ 멋있어요\z여친 ...
5,"(경돼 유도) 주말에 유도 한판, 청림유도관",2085,2021.3.7,39,0,주말 유도,,1:23,15,경돼형 유투브 다시 시작했구나!!!!!\z경돼님 유도영상 너무 좋아요\z설킴이 응원...
6,출퇴근 길은 헬스장으로,3563,2021.3.1,89,1,여러분 안녕하세요 경돼입니다.\n\n\n\n----------------------...,,8:47,43,경돼 영상으로 주말 마무리 하는거 찐소확행\z같은 동네 살아서 배경이 친근하네요\n...
7,천천히 운동 쌓아 나가기│브이로그,3563,2021.2.21,90,3,여러분 안녕하세요 경돼입니다.\n\n\n\n----------------------...,,0:30,37,경돼형님 벌써 4년가까이 구독중인데 항상 영상재밌게 보구있습니다 ㅋㅋ\n운동도 그렇...
8,"(체중78kg) 퇴근 후 벤치프레스 100kg, 아슬아슬",4725,2021.2.21,49,0,여러분 안녕하세요 경돼입니다.\n\n\n\n----------------------...,,0:34,21,크 경돼님 추억이네. 군대 전역하고 대학교 3학년 때 처음 경돼랑 운지기 보면서 이...
9,"3개월 만에 찾은 헬스장, 회사원VLOG",3209,2021.2.12,106,5,여러분 안녕하세요 경돼입니다.\n\n즐거운 설 연휴 보내세요!\n\nBe more ...,,17:21,35,그저 평범한 사회의 한 구성원으로서 \n가장 현실적인 모습을 보여주는 \n이시대 최...


- 댓글수 : html로 받기
- 태그도 오류 예외문 작성 필요
- 광고 고려 필요
    - 광고 길이가 긴 영상들이 있음